Beat-n game with 16 pt limit

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const ability_none = 999
const ability_imitative = 10
const ability_reciprocating = 20
const ability_romantic = 30
const ability_bibliophile = 40
const ability_melodious = 50
const ability_iconoclast = 90
const ability_intoxicating = 100
const ability_otaku = 110
const ability_sincere = 120
const ability_cowardly = 130


ability_name = Dict(
    ability_imitative => "Imitative*",
    ability_reciprocating => "Reciprocating*",
    ability_romantic => "Romantic*",
    ability_bibliophile => "Bibliophile*",
    ability_melodious => "Melodious*",
    ability_iconoclast => "Iconoclast*",
    ability_intoxicating => "Intoxicating*",
    ability_otaku => "Otaku*",
    ability_sincere => "Sincere*",
    ability_cowardly => "Cowardly*",
)

function fx_imitative(a::Spud, b::Spud)
    if b.h > a.h
        a = Spud(a.name, b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_reciprocating(a::Spud, b::Spud)
    if b.f > a.f
        a = Spud(a.name, a.h, b.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_romantic(a::Spud, b::Spud)
    if b.p > a.l
        a = Spud(a.name, a.h, a.f, b.p, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_bibliophile(a::Spud, b::Spud)
    if b.r > a.l
        a = Spud(a.name, a.h, a.f, b.r, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_melodious(a::Spud, b::Spud)
    if b.p > b.r && b.p > b.s
        a = Spud(a.name, a.h, a.f, a.r, a.p + 3, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_iconoclast(a::Spud, b::Spud)
    v = sum([b.h,b.f,b.l,b.p,b.r,b.s] .== MXV)
    if v >= 3
        aa = [a.h, a.f, a.l, a.p, a.r, a.s]
        aa2 = map(x -> div(x, 4), aa)
        r = aa .+ aa2
        a = Spud(a.name, r[1], r[2], r[3], r[4], r[5], r[6], a.a1)
    end
    return [a, b]
end

function fx_intoxicating(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.f, b.a1)
    return [a, b]
end

function fx_otaku(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.h, b.s, b.a1)
    return [a, b]
end


function fx_sincere(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f + 2, b.f + 2, b.p, b.r, b.s, b.a1)
    return [a, b]
end

function fx_cowardly(a::Spud, b::Spud)
    if (a.p < b.p) || (a.r < b.r) || (a.s < b.s)
        a = Spud(a.name, a.h + 2, a.f - 2, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

ability_fx = Dict(
    ability_imitative => fx_imitative,
    ability_reciprocating => fx_reciprocating,
    ability_romantic => fx_romantic,
    ability_bibliophile => fx_bibliophile,
    ability_melodious => fx_melodious,
    ability_iconoclast => fx_iconoclast,
    ability_intoxicating => fx_intoxicating,
    ability_otaku => fx_otaku,
    ability_sincere => fx_sincere,
    ability_cowardly => fx_cowardly,
)

Dict{Int64, Function} with 10 entries:
  50  => fx_melodious
  20  => fx_reciprocating
  110 => fx_otaku
  10  => fx_imitative
  90  => fx_iconoclast
  30  => fx_romantic
  120 => fx_sincere
  130 => fx_cowardly
  40  => fx_bibliophile
  100 => fx_intoxicating

In [4]:
#const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

1

In [5]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [6]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [7]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)


const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat(ability::Int64 = ability_none)::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    if ability == ability_none
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
        vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
        vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
        vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
        vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
        vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    else
        adj = ability_name[ability]
    end
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6], ability_none)
end

random_name_and_stat (generic function with 2 methods)

In [8]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat(a.a1)
    for ii in 1:n_tries
        b = random_name_and_stat(a.a1)
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s, a.a1)
end

rand_rename (generic function with 2 methods)

In [9]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get spuds with cost 20

In [10]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff = Spud("", h,f,l,p,r,s,999)
                                ff_i += 1
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 2 methods)

In [11]:
lib0 = get_library(16, 20000)
lib0

2967-element Vector{Spud}:
 Spud("", 1, 1, 1, 1, 3, 9, 999)
 Spud("", 1, 1, 1, 1, 4, 8, 999)
 Spud("", 1, 1, 1, 1, 5, 7, 999)
 Spud("", 1, 1, 1, 1, 6, 6, 999)
 Spud("", 1, 1, 1, 1, 7, 5, 999)
 Spud("", 1, 1, 1, 1, 8, 4, 999)
 Spud("", 1, 1, 1, 1, 9, 3, 999)
 Spud("", 1, 1, 1, 2, 2, 9, 999)
 Spud("", 1, 1, 1, 2, 3, 8, 999)
 Spud("", 1, 1, 1, 2, 4, 7, 999)
 Spud("", 1, 1, 1, 2, 5, 6, 999)
 Spud("", 1, 1, 1, 2, 6, 5, 999)
 Spud("", 1, 1, 1, 2, 7, 4, 999)
 ⋮
 Spud("", 9, 1, 1, 2, 1, 2, 999)
 Spud("", 9, 1, 1, 2, 2, 1, 999)
 Spud("", 9, 1, 1, 3, 1, 1, 999)
 Spud("", 9, 1, 2, 1, 1, 2, 999)
 Spud("", 9, 1, 2, 1, 2, 1, 999)
 Spud("", 9, 1, 2, 2, 1, 1, 999)
 Spud("", 9, 1, 3, 1, 1, 1, 999)
 Spud("", 9, 2, 1, 1, 1, 2, 999)
 Spud("", 9, 2, 1, 1, 2, 1, 999)
 Spud("", 9, 2, 1, 2, 1, 1, 999)
 Spud("", 9, 2, 2, 1, 1, 1, 999)
 Spud("", 9, 3, 1, 1, 1, 1, 999)

In [12]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [13]:
function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

filter_nondominated2 (generic function with 1 method)

In [14]:
lib=filter_nondominated2(lib0)
for ff in lib
    print(" ")
    print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
end
# lib = lib0;

 111139 111148 111157 111166 111175 111184 111193 111229 111238 111247 111256 111265 111274 111283 111292 111319 111328 111337 111346 111355 111364 111373 111382 111391 111418 111427 111436 111445 111454 111463 111472 111481 111517 111526 111535 111544 111553 111562 111571 111616 111625 111634 111643 111652 111661 111715 111724 111733 111742 111751 111814 111823 111832 111841 111913 111922 111931 112129 112138 112147 112156 112165 112174 112183 112192 112219 112228 112237 112246 112255 112264 112273 112282 112291 112318 112327 112336 112345 112354 112363 112372 112381 112417 112426 112435 112444 112453 112462 112471 112516 112525 112534 112543 112552 112561 112615 112624 112633 112642 112651 112714 112723 112732 112741 112813 112822 112831 112912 112921 113119 113128 113137 113146 113155 113164 113173 113182 113191 113218 113227 113236 113245 113254 113263 113272 113281 113317 113326 113335 113344 113353 113362 113371 113416 113425 113434 113443 113452 113461 113515 113524 113533 11354

 315232 315241 315313 315322 315331 315412 315421 315511 316114 316123 316132 316141 316213 316222 316231 316312 316321 316411 317113 317122 317131 317212 317221 317311 318112 318121 318211 319111 322117 322126 322135 322144 322153 322162 322171 322216 322225 322234 322243 322252 322261 322315 322324 322333 322342 322351 322414 322423 322432 322441 322513 322522 322531 322612 322621 322711 323116 323125 323134 323143 323152 323161 323215 323224 323233 323242 323251 323314 323323 323332 323341 323413 323422 323431 323512 323521 323611 324115 324124 324133 324142 324151 324214 324223 324232 324241 324313 324322 324331 324412 324421 324511 325114 325123 325132 325141 325213 325222 325231 325312 325321 325411 326113 326122 326131 326212 326221 326311 327112 327121 327211 332116 332125 332134 332143 332152 332215 332224 332233 332242 332251 332314 332323 332332 332341 332413 332422 332431 332512 332521 332611 333115 333124 333133 333142 333214 333223 333232 333241 333313 333322 333331 33341

In [15]:
## iterating doesn't change
# lib=filter_nondominated2(lib)
# for ff in lib
#     print(" ")
#     print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
# end

In [16]:
length(lib)

2112

In [17]:
#payoffs = get_payoffs(lib)

## Battle code

In [18]:
mutable struct BattleView
    round::Int64
    score::Int64
    own::Array{Spud}
    obs::Array{Spud}
end

mutable struct AiPlayer
    create::Function
end

In [19]:
function initialize_game()::Array{BattleView}
    bvs = [BattleView(0,0, [], []), BattleView(0,0, [], [])]
    return bvs
end

function spud_to_str(ff::Spud)::String
    return string(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
end

function eval_beat_n(ff::Spud, opps::Array{Spud})::Int64
    flag = true
    count = 0
    while flag && count < length(opps)
        b = opps[end - count]
        if eval_battle(ff, b) == 1
            count += 1
        else
            flag = false
        end
    end
    return count
end

eval_beat_n (generic function with 1 method)

In [20]:
# occurs after spud creation
function play_combat(game::Array{BattleView}, verbose::Bool = false)::Array{BattleView}
    round = game[1].round
    if mod(round, 2)==1
        ff = game[1].own[end]
        opps = game[2].own
    else
        ff = game[2].own[end]
        opps = game[1].own
    end
    count = eval_beat_n(ff, opps)
    if verbose
        print(spud_to_str(ff))
        print(" beats ")
        println([spud_to_str(b) for b in opps[(end-count+1):end]])
    end
    if mod(round, 2)==1
        game[1].score = count
    else
        game[2].score = count
    end
    return game
end

function play_add_spud(game::Array{BattleView}, player::Int64, ff::Spud, verbose::Bool = false)::Array{BattleView}
    round = game[1].round + 1
    game[1].round = round
    game[2].round = round
    game[player].own = append!(game[player].own, [ff])
    game[3-player].obs = game[player].own
    if verbose
        print("Round ")
        println(round)
        print("Score to match or beat: ")
        println(max(game[1].score, game[2].score))
        print("Player ")
        print(player)
        print(" adds #")
        print(["A", "B"][player])
        print(length(game[player].own))
        print(" ")
        println(spud_to_str(ff))
    end
    return game
end
    
    

function play_is_game_over(game::Array{BattleView})::Bool
    round = game[1].round
    if mod(round, 2)==1
        if game[1].score < game[2].score
            return true
        end
    else
        if game[2].score < game[1].score
            return true
        end        
    end
    return false
end

play_is_game_over (generic function with 1 method)

In [21]:
function ai0_create(a::BattleView)::Spud
    return rand(lib)
end

function highest_beat_n(ops::Array{Spud})::Array{Spud}
    ebs = [eval_beat_n(ff, ops) for ff in lib]
    cands = lib[ebs .== maximum(ebs)]
    return cands
end

function ai1_create(a::BattleView)::Spud
    ops = a.obs
    cands = highest_beat_n(ops)
    return rand(cands)
end


ai1_create (generic function with 1 method)

In [22]:
function input_spud()::Spud
    s = readline()
    return stats_to_spud(s)
end

function stats_to_spud(s::String)::Spud
    v = Array{Int64}(undef, 6)
    for i in 1:6
        v[i] = convert(Int64, s[i]) - 48
    end
    return Spud("", v[1], v[2], v[3], v[4], v[5], v[6], 999)
end

function player_create(a::BattleView)::Spud
    return input_spud()
end

player_create (generic function with 1 method)

In [23]:
function play_game(players::Array{AiPlayer}, verbose::Bool = true)::Int64
    ai1 = players[1]
    ai2 = players[2]
    ais = players
    game = initialize_game()
    while !play_is_game_over(game)
        # play a round
        round = game[1].round
        if mod(round, 2)==0
            ff = ais[1].create(game[1])
            game = play_add_spud(game, 1, ff, verbose)
        else
            ff = ais[2].create(game[2])
            game = play_add_spud(game, 2, ff, verbose)
        end
        if round > 0
            game = play_combat(game, verbose)
        else
            println("First round: no combat.")
        end
    end
    if verbose
        println("Game history:")
        for i in 1:length(game[1].own)
            print(spud_to_str(game[1].own[i]))
            print(" ")
            if i <= length(game[2].own)
                println(spud_to_str(game[2].own[i]))
            end
        end
    end
    if game[1].score > game[2].score
        return 1
    else
        return -1
    end
end

play_game (generic function with 2 methods)

In [24]:
function play_game_n(players::Array{AiPlayer}, n::Int64)
    verbose = true
    ai1 = players[1]
    ai2 = players[2]
    ais = players
    game = initialize_game()
    for i in 1:n
        # play a round
        round = game[1].round
        if mod(round, 2)==0
            ff = ais[1].create(game[1])
            game = play_add_spud(game, 1, ff, verbose)
        else
            ff = ais[2].create(game[2])
            game = play_add_spud(game, 2, ff, verbose)
        end
        if round > 0
            game = play_combat(game, verbose)
        else
            println("First round: no combat.")
        end
    end
    if verbose
        println("Game history:")
        for i in 1:length(game[1].own)
            print(spud_to_str(game[1].own[i]))
            print(" ")
            if i <= length(game[2].own)
                println(spud_to_str(game[2].own[i]))
            end
        end
    end
end

play_game_n (generic function with 1 method)

In [25]:
ai1 = AiPlayer(ai1_create)
ai2 = AiPlayer(ai1_create)
ais = [ai1, ai2]
play_game(ais, true)

Round 1
Score to match or beat: 0
Player 1 adds #A1 312181
First round: no combat.
Round 2
Score to match or beat: 0
Player 2 adds #B1 117124
117124 beats ["312181"]
Round 3
Score to match or beat: 1
Player 1 adds #A2 132325
132325 beats ["117124"]
Round 4
Score to match or beat: 1
Player 2 adds #B2 313315
313315 beats ["312181", "132325"]
Round 5
Score to match or beat: 2
Player 1 adds #A3 131317
131317 beats ["117124", "313315"]
Round 6
Score to match or beat: 2
Player 2 adds #B3 433132
433132 beats ["312181", "132325", "131317"]
Round 7
Score to match or beat: 3
Player 1 adds #A4 124423
124423 beats ["117124", "313315", "433132"]
Round 8
Score to match or beat: 3
Player 2 adds #B4 435112
435112 beats ["312181", "132325", "131317", "124423"]
Round 9
Score to match or beat: 4
Player 1 adds #A5 151414
151414 beats ["117124", "313315", "433132", "435112"]
Round 10
Score to match or beat: 4
Player 2 adds #B5 131722
131722 beats ["312181", "132325", "131317", "124423", "151414"]
Round 11


-1

In [26]:
ai1 = AiPlayer(ai1_create)
ai2 = AiPlayer(ai1_create)
ais = [ai1, ai2]
#play_game(ais, true)
play_game_n(ais, 1000)

Round 1
Score to match or beat: 0
Player 1 adds #A1 144313
First round: no combat.
Round 2
Score to match or beat: 0
Player 2 adds #B1 215341
215341 beats ["144313"]
Round 3
Score to match or beat: 1
Player 1 adds #A2 246112
246112 beats ["215341"]
Round 4
Score to match or beat: 1
Player 2 adds #B2 132415
132415 beats ["144313", "246112"]
Round 5
Score to match or beat: 2
Player 1 adds #A3 133612
133612 beats ["215341", "132415"]
Round 6
Score to match or beat: 2
Player 2 adds #B3 141127
141127 beats ["144313", "246112", "133612"]
Round 7
Score to match or beat: 3
Player 1 adds #A4 141541
141541 beats ["215341", "132415", "141127"]
Round 8
Score to match or beat: 3
Player 2 adds #B4 121453
121453 beats ["144313", "246112", "133612", "141541"]
Round 9
Score to match or beat: 4
Player 1 adds #A5 222145
222145 beats ["215341", "132415", "141127", "121453"]
Round 10
Score to match or beat: 4
Player 2 adds #B5 131254
131254 beats ["144313", "246112", "133612", "141541", "222145"]
Round 11


216151 beats ["151144", "414412", "616111", "415231", "115315", "125341", "135331"]
Round 75
Score to match or beat: 7
Player 1 adds #A38 216241
216241 beats ["615211", "162124", "116134", "116125", "216124", "216151"]
Round 76
Score to match or beat: 7
Player 2 adds #B38 116314
116314 beats ["151144", "414412", "616111", "415231", "115315", "125341", "135331", "216241"]
Round 77
Score to match or beat: 8
Player 1 adds #A39 218131
218131 beats ["615211", "162124", "116134", "116125", "216124", "216151", "116314"]
Round 78
Score to match or beat: 8
Player 2 adds #B39 218311
218311 beats ["414412", "616111", "415231", "115315", "125341", "135331", "216241", "218131"]
Round 79
Score to match or beat: 8
Player 1 adds #A40 125215
125215 beats ["615211", "162124", "116134", "116125", "216124", "216151", "116314", "218311"]
Round 80
Score to match or beat: 8
Player 2 adds #B40 161413
161413 beats ["414412", "616111", "415231", "115315", "125341", "135331", "216241", "218131", "125215"]
Round 

218131 beats ["213424", "715111", "616111", "117214", "317122", "117133", "228112"]
Round 130
Score to match or beat: 7
Player 2 adds #B65 219112
219112 beats ["813121", "714112", "115423", "517111", "217411", "117421", "218113", "218131"]
Round 131
Score to match or beat: 8
Player 1 adds #A66 219121
219121 beats ["213424", "715111", "616111", "117214", "317122", "117133", "228112", "219112"]
Round 132
Score to match or beat: 8
Player 2 adds #B66 219211
219211 beats ["813121", "714112", "115423", "517111", "217411", "117421", "218113", "218131", "219121"]
Round 133
Score to match or beat: 9
Player 1 adds #A67 171331
171331 beats ["715111", "616111", "117214", "317122", "117133", "228112", "219112", "219211"]
Round 134
Score to match or beat: 9
Player 2 adds #B67 151513
151513 beats ["115423", "517111", "217411", "117421", "218113", "218131", "219121", "171331"]
Round 135
Score to match or beat: 8
Player 1 adds #A68 161215
161215 beats ["616111", "117214", "317122", "117133", "228112", 

Round 180
Score to match or beat: 8
Player 2 adds #B90 121615
121615 beats ["141325", "141442", "141154", "151522", "113524", "114523", "214513", "215143", "216133"]
Round 181
Score to match or beat: 9
Player 1 adds #A91 516211
516211 beats ["141523", "543112", "642121", "624211", "111166", "615112", "616111", "121615"]
Round 182
Score to match or beat: 9
Player 2 adds #B91 116134
116134 beats ["141154", "151522", "113524", "114523", "214513", "215143", "216133", "516211"]
Round 183
Score to match or beat: 8
Player 1 adds #A92 517111
517111 beats ["141523", "543112", "642121", "624211", "111166", "615112", "616111", "121615", "116134"]
Round 184
Score to match or beat: 9
Player 2 adds #B92 217132
217132 beats ["151522", "113524", "114523", "214513", "215143", "216133", "516211", "517111"]
Round 185
Score to match or beat: 9
Player 1 adds #A93 317212
317212 beats ["642121", "624211", "111166", "615112", "616111", "121615", "116134", "217132"]
Round 186
Score to match or beat: 8
Player 2

Round 243
Score to match or beat: 7
Player 1 adds #A122 161512
161512 beats ["171142", "161332", "228211", "319111", "116251", "516121", "616111", "123415"]
Round 244
Score to match or beat: 8
Player 2 adds #B122 121525
121525 beats ["215224", "113155", "223144", "232144", "141415", "151342", "162322", "161512"]
Round 245
Score to match or beat: 8
Player 1 adds #A123 161521
161521 beats ["228211", "319111", "116251", "516121", "616111", "123415", "121525"]
Round 246
Score to match or beat: 8
Player 2 adds #B123 122515
122515 beats ["215224", "113155", "223144", "232144", "141415", "151342", "162322", "161512", "161521"]
Round 247
Score to match or beat: 9
Player 1 adds #A124 418111
418111 beats ["116251", "516121", "616111", "123415", "121525", "122515"]
Round 248
Score to match or beat: 9
Player 2 adds #B124 152512
152512 beats ["141415", "151342", "162322", "161512", "161521", "418111"]
Round 249
Score to match or beat: 6
Player 1 adds #A125 317122
317122 beats ["116251", "516121", "

...

Round 777
Score to match or beat: 9
Player 1 adds #A389 814111
814111 beats ["121714", "121525", "633112", "632212", "141145", "151441", "151432", "161215", "161431", "163231"]
Round 778
Score to match or beat: 10
Player 2 adds #B389 219121
219121 beats ["319111", "318121", "317122", "517111", "525112", "616111", "513223", "613222", "814111"]
Round 779
Score to match or beat: 10
Player 1 adds #A390 123415
123415 beats ["633112", "632212", "141145", "151441", "151432", "161215", "161431", "163231", "219121"]
Round 780
Score to match or beat: 9
Player 2 adds #B390 318211
318211 beats ["318121", "317122", "517111", "525112", "616111", "513223", "613222", "814111", "123415"]
Round 781
Score to match or beat: 9
Player 1 adds #A391 132253
132253 beats ["632212", "141145", "151441", "151432", "161215", "161431", "163231", "219121", "318211"]
Round 782
Score to match or beat: 9
Player 2 adds #B391 418111
418111 beats ["317122", "517111", "525112", "616111", "513223", "613222", "814111", "12341

122641 beats ["151513", "132514", "114523", "212542", "112525", "121525", "162232"]
Round 832
Score to match or beat: 8
Player 2 adds #B416 161242
161242 beats ["161233", "171511", "161611", "161125", "116125", "216124", "616111", "122641"]
Round 833
Score to match or beat: 8
Player 1 adds #A417 112633
112633 beats ["151513", "132514", "114523", "212542", "112525", "121525", "162232", "161242"]
Round 834
Score to match or beat: 8
Player 2 adds #B417 316141
316141 beats ["171511", "161611", "161125", "116125", "216124", "616111", "122641", "112633"]
Round 835
Score to match or beat: 8
Player 1 adds #A418 131623
131623 beats ["151513", "132514", "114523", "212542", "112525", "121525", "162232", "161242", "316141"]
Round 836
Score to match or beat: 9
Player 2 adds #B418 416131
416131 beats ["171511", "161611", "161125", "116125", "216124", "616111", "122641", "112633", "131623"]
Round 837
Score to match or beat: 9
Player 1 adds #A419 416212
416212 beats ["132514", "114523", "212542", "112

417211 beats ["232252", "133153", "131353", "131425", "133324", "814111", "113524", "123514", "415114", "616111", "517111"]
Round 891
Score to match or beat: 11
Player 1 adds #A446 318211
318211 beats ["613213", "122515", "114433", "114415", "424411", "415141", "416221", "417211"]
Round 892
Score to match or beat: 11
Player 2 adds #B446 319111
319111 beats ["814111", "113524", "123514", "415114", "616111", "517111", "318211"]
Round 893
Score to match or beat: 8
Player 1 adds #A447 142513
142513 beats ["122515", "114433", "114415", "424411", "415141", "416221", "417211", "319111"]
Round 894
Score to match or beat: 8
Player 2 adds #B447 161125
161125 beats ["113524", "123514", "415114", "616111", "517111", "318211", "142513"]
Round 895
Score to match or beat: 8
Player 1 adds #A448 141532
141532 beats ["114433", "114415", "424411", "415141", "416221", "417211", "319111", "161125"]
Round 896
Score to match or beat: 8
Player 2 adds #B448 161134
161134 beats ["616111", "517111", "318211", "1

Round 944
Score to match or beat: 6
Player 2 adds #B472 352213
352213 beats ["131641", "516211", "418111", "452122", "452212", "151216"]
Round 945
Score to match or beat: 6
Player 1 adds #A473 153124
153124 beats ["516121", "416311", "318112", "453112", "151144", "352213"]
Round 946
Score to match or beat: 6
Player 2 adds #B473 151243
151243 beats ["131641", "516211", "418111", "452122", "452212", "151216", "153124"]
Round 947
Score to match or beat: 7
Player 1 adds #A474 152215
152215 beats ["516121", "416311", "318112", "453112", "151144", "352213", "151243"]
Round 948
Score to match or beat: 7
Player 2 adds #B474 151342
151342 beats ["131641", "516211", "418111", "452122", "452212", "151216", "153124", "152215"]
Round 949
Score to match or beat: 8
Player 1 adds #A475 152314
152314 beats ["516121", "416311", "318112", "453112", "151144", "352213", "151243", "151342"]
Round 950
Score to match or beat: 8
Player 2 adds #B475 151522
151522 beats ["516211", "418111", "452122", "452212", "

Round 999
Score to match or beat: 10
Player 1 adds #A500 552112
552112 beats ["417112", "417211", "418111", "141226", "141541", "151441", "141442"]
Round 1000
Score to match or beat: 10
Player 2 adds #B500 142342
142342 beats ["315142", "225115", "417121", "327112", "318112", "141325", "151342", "151621", "552112"]
Game history:
144313 215341
246112 132415
133612 141127
141541 121453
222145 131254
121561 523123
616111 516121
416131 517111
417211 327211
252214 317221
418111 263212
152224 262222
161314 161332
151414 161422
151144 141451
151153 141424
131542 111166
232441 122515
513412 113155
715111 315124
516211 326113
517111 317113
131236 417121
242215 117241
319111 171331
141361 161431
142252 152332
141253 543112
143422 132343
151513 131533
151144 132334
414412 615211
616111 162124
415231 116134
115315 116125
125341 216124
135331 216151
216241 116314
218131 218311
125215 161413
222325 126322
223234 141433
122524 131425
132442 132541
131452 132532
121156 324412
131623 323314
141631 4243

In [27]:
# ai1 = AiPlayer(ai1_create)
# ai2 = AiPlayer(player_create)
# ais = [ai2, ai1]
# game = play_game(ais, true)

In [28]:
# highest_beat_n([stats_to_spud(s) for s in ["113311", "213211", "213112", "121132"]])